
# Modelado Predictivo del Rendimiento Financiero en Unidades de Negocio mediante Machine Learning


# Fuentes de datos y unificación

En este proyecto se utilizan **tres fuentes de datos sintéticas** que simulan distintos sistemas internos de una organización:

- **Datos financieros**: información económica básica de las unidades de negocio (ingresos, gastos, activos, pasivos, EBIT).
- **Ratios financieros**: indicadores derivados del reporting contable (ROA, ROE, margen de explotación, endeudamiento, índice de rentabilidad).
- **Datos operativos**: variables relacionadas con estructura operativa y costes (plantilla, costes fijos y variables, crecimiento de ingresos).

Los datasets son **sintéticos**, generados para preservar la confidencialidad, pero diseñados para mantener **relaciones económicas plausibles** entre las variables, reproduciendo escenarios habituales en entornos corporativos reales.

El objetivo de este notebook es **construir un dataset coherente y estandarizado** a partir de estas tres fuentes heterogéneas, que presentan diferencias en:
- nombres de columnas,
- formatos numéricos,
- y presencia de valores faltantes.

A lo largo de este notebook se realizan las siguientes tareas:

- Carga de los datasets originales en formato CSV.
- Revisión de estructuras, tipos de datos e identificación de inconsistencias.
- Estandarización de nombres de columnas y formatos numéricos.
- Tratamiento **mínimo** de valores nulos necesario para poder unificar las fuentes.
- Unificación de las fuentes mediante claves comunes (`Unidad_ID`, `Periodo_ID`).
- Generación de un dataset consolidado que servirá como punto de partida para el análisis exploratorio y el modelado posterior.


In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from sklearn.metrics import r2_score


# 1- Carga de datos 


In [56]:
fin = pd.read_csv(r"C:\Users\balle\Desktop\DataScience\ML_project\data\Raw\raw_financials.csv")
rat = pd.read_csv(r"C:\Users\balle\Desktop\DataScience\ML_project\data\Raw\raw_ratios.csv")
ops = pd.read_csv(r"C:\Users\balle\Desktop\DataScience\ML_project\data\Raw\raw_operations.csv")

> Nota: Los datos utilizados en este proyecto son **sintéticos**, generados para simular sistemas internos reales y preservar la confidencialidad.  
> Se han diseñado para mantener relaciones financieras y operativas plausibles entre variables.


## 1.1- Revisión de los datos 



In [57]:
fin.head(5)

,Unidad_ID,Sector,Periodo_ID,IngRsos,Gastos,Activos,Pasivos,EBIT
0,UO_086,Logística,1,"123456,78",24987.379399,10860.717125,9234.423602,3811.366786
1,UO_015,Tecnología,8,30450.09122566627,19446.985846,17670.376360,12420.632727,12315.649015
2,UO_015,Tecnología,8,30450.09122566627,19446.985846,17670.376360,12420.632727,12315.649015
3,UO_077,Finanzas,1,47170.07320439235,33076.391477,6926.752898,2096.144509,13716.391078
4,UO_001,Tecnología,9,19592.915242189152,13880.256093,23211.886570,19841.303916,6666.729731


In [58]:
rat.head(5)

,Unidad_ID,Periodo_ID,R.O.A,ROE,Margen_Explotacion,Endeudamiento,Indice_Rentabilidad
0,UO_082,6,0.087238,0.007010,0.115056,0.850259,-0.103534
1,UO_120,4,-0.019214,0.006369,0.160203,0.380385,-0.034358
2,UO_047,2,0.082736,0.119601,0.361349,0.702907,0.058364
3,UO_107,7,0.056002,-0.034820,0.298784,0.302616,0.039246
4,UO_025,2,0.070150,0.190301,0.291568,0.854791,-0.050394


In [59]:
ops.head(5)

,Unidad_ID,Periodo_ID,Plantilla,Costes_fij,CostesVariables,Crecimiento_Ingresos
0,UO_094,1,283,9296.337623,11160.394790,0.278319
1,UO_105,10,422,7028.864683,9194.617841,0.073097
2,UO_119,1,227,5104.524438,7593.045656,0.048345
3,UO_035,6,471,13751.734818,10470.481717,0.262840
4,UO_071,6,161,5222.166774,5128.478498,-0.052685


In [60]:
fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unidad_ID   2500 non-null   object 
 1   Sector      2500 non-null   object 
 2   Periodo_ID  2500 non-null   int64  
 3   IngRsos     2500 non-null   object 
 4   Gastos      2500 non-null   float64
 5   Activos     2500 non-null   float64
 6   Pasivos     2475 non-null   float64
 7   EBIT        2500 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 156.4+ KB


In [61]:
rat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unidad_ID            2500 non-null   object 
 1   Periodo_ID           2500 non-null   int64  
 2   R.O.A                2500 non-null   float64
 3   ROE                  2500 non-null   float64
 4   Margen_Explotacion   2500 non-null   float64
 5   Endeudamiento        2500 non-null   float64
 6   Indice_Rentabilidad  2480 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 136.8+ KB


In [62]:
ops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unidad_ID             2500 non-null   object 
 1   Periodo_ID            2500 non-null   int64  
 2   Plantilla             2500 non-null   object 
 3   Costes_fij            2480 non-null   float64
 4   CostesVariables       2500 non-null   float64
 5   Crecimiento_Ingresos  2500 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 117.3+ KB


# 2- Detección de los valores nulos  


### Tratamiento mínimo de valores nulos

En esta fase se realiza únicamente un tratamiento instrumental de valores nulos, con el objetivo de permitir la integración de las distintas fuentes y preservar la estructura del dataset.

Estas imputaciones no se consideran definitivas desde el punto de vista analítico.
Las estrategias de imputación finales se definen posteriormente en el EDA, donde se analiza el comportamiento de las variables por departamento y su impacto en el modelado.


In [63]:
print("Nulos en Financials:\n", fin.isnull().sum(), "\n")
print("Nulos en Ratios:\n", rat.isnull().sum(), "\n")
print("Nulos en Operations:\n", ops.isnull().sum(), "\n")


Nulos en Financials:
 Unidad_ID      0
Sector         0
Periodo_ID     0
IngRsos        0
Gastos         0
Activos        0
Pasivos       25
EBIT           0
dtype: int64 

Nulos en Ratios:
 Unidad_ID               0
Periodo_ID              0
R.O.A                   0
ROE                     0
Margen_Explotacion      0
Endeudamiento           0
Indice_Rentabilidad    20
dtype: int64 

Nulos en Operations:
 Unidad_ID                0
Periodo_ID               0
Plantilla                0
Costes_fij              20
CostesVariables          0
Crecimiento_Ingresos     0
dtype: int64 



In [64]:
# Tengo valores nulos en la columna "pasivos", remplazo por la mediana
# ya que considero que es lo más apropiado para estos valores del balance
fin["Pasivos"] = fin["Pasivos"].fillna(fin["Pasivos"].median())

In [65]:
print("Nulos en Financials:\n", fin.isnull().sum(), "\n")

Nulos en Financials:
 Unidad_ID     0
Sector        0
Periodo_ID    0
IngRsos       0
Gastos        0
Activos       0
Pasivos       0
EBIT          0
dtype: int64 



In [66]:
# De igual manera con los nulos encontrados en ratios, sustituyo por mediana
rat["Indice_Rentabilidad"] = rat["Indice_Rentabilidad"].fillna(rat["Indice_Rentabilidad"].median())

In [67]:
print("Nulos en Ratios:\n", rat.isnull().sum(), "\n")

Nulos en Ratios:
 Unidad_ID              0
Periodo_ID             0
R.O.A                  0
ROE                    0
Margen_Explotacion     0
Endeudamiento          0
Indice_Rentabilidad    0
dtype: int64 



In [68]:
# En costes_fijos los valores nulos los sustituyo por la media porque 
# es un coste proporcional y ya viene de Gastos de la PyG
ops["Costes_fij"] = ops["Costes_fij"].fillna(ops["Costes_fij"].mean())

In [69]:
print("Nulos en Operations:\n", ops.isnull().sum(), "\n")

Nulos en Operations:
 Unidad_ID               0
Periodo_ID              0
Plantilla               0
Costes_fij              0
CostesVariables         0
Crecimiento_Ingresos    0
dtype: int64 



# 3 Renombrar columnas 

In [70]:
fin = fin.rename(columns={"IngRsos": "Ingresos"})
fin = fin.rename(columns={"IngRsos": "Ingresos", "Sector": "Departamento"})
rat = rat.rename(columns={"R.O.A": "ROA"})
ops = ops.rename(columns={"Costes_fij": "Costes_Fijos"})

In [71]:
fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unidad_ID     2500 non-null   object 
 1   Departamento  2500 non-null   object 
 2   Periodo_ID    2500 non-null   int64  
 3   Ingresos      2500 non-null   object 
 4   Gastos        2500 non-null   float64
 5   Activos       2500 non-null   float64
 6   Pasivos       2500 non-null   float64
 7   EBIT          2500 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 156.4+ KB


In [72]:
rat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unidad_ID            2500 non-null   object 
 1   Periodo_ID           2500 non-null   int64  
 2   ROA                  2500 non-null   float64
 3   ROE                  2500 non-null   float64
 4   Margen_Explotacion   2500 non-null   float64
 5   Endeudamiento        2500 non-null   float64
 6   Indice_Rentabilidad  2500 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 136.8+ KB


In [73]:
ops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unidad_ID             2500 non-null   object 
 1   Periodo_ID            2500 non-null   int64  
 2   Plantilla             2500 non-null   object 
 3   Costes_Fijos          2500 non-null   float64
 4   CostesVariables       2500 non-null   float64
 5   Crecimiento_Ingresos  2500 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 117.3+ KB


# 4 Convertir tipos de datos

### Normalización de formatos y tipos de datos

Las fuentes presentan diferencias habituales en entornos reales (formatos decimales, columnas numéricas como texto).  
En esta fase se corrigen únicamente problemas de formato para asegurar consistencia estructural.


#### 4.1 Convertir columnas object a float

In [74]:
# De la columna empleados, extraigo solo el nº de empleados y convierto a float
ops["Plantilla"] = ops["Plantilla"].astype(str).str.extract("(\d+)").astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\balle\AppData\Local\Temp\ipykernel_13568\2632162056.py:2: SyntaxWarning: invalid escape sequence '\d'
  ops["Plantilla"] = ops["Plantilla"].astype(str).str.extract("(\d+)").astype(float)


#### 4.2 Convertir números con coma decimal a punto decimal

In [75]:
fin["Ingresos"] = fin["Ingresos"].astype(str).str.replace(",", ".", regex=False)
fin["Ingresos"] = pd.to_numeric(fin["Ingresos"], errors="coerce")

In [82]:
# Revisión todas las columnas de datos numéricos

num_cols_fin = ["Ingresos", "Gastos", "Activos", "Pasivos", "EBIT"]
num_cols_rat = ["ROA", "ROE", "Margen_Explotacion", "Endeudamiento", "Indice_Rentabilidad"]
num_cols_ops = ["Plantilla", "Costes_Fijos", "CostesVariables", "Crecimiento_Ingresos"]

for col in num_cols_fin:
    if col in fin.columns:
        fin[col] = pd.to_numeric(fin[col], errors="coerce")

for col in num_cols_rat:
    if col in rat.columns:
        rat[col] = pd.to_numeric(rat[col], errors="coerce")

for col in num_cols_ops:
    if col in ops.columns:
        ops[col] = pd.to_numeric(ops[col], errors="coerce")

## 5 Eliminación de duplicados

In [83]:

fin = fin.drop_duplicates()
rat = rat.drop_duplicates()
ops = ops.drop_duplicates()


In [84]:
fin = fin.drop_duplicates(subset=["Unidad_ID", "Periodo_ID"], keep="first")
rat = rat.drop_duplicates(subset=["Unidad_ID", "Periodo_ID"], keep="first")
ops = ops.drop_duplicates(subset=["Unidad_ID", "Periodo_ID"], keep="first")


## 6 Merge de los csv

In [85]:
print("Filas Financials:", len(fin))
print("Filas Ratios:", len(rat))
print("Filas Operations:", len(ops))

Filas Financials: 1041
Filas Ratios: 1057
Filas Operations: 1058


Aunque cada fuente contiene 2500 registros, no todas las combinaciones de `Unidad_ID` y `Periodo_ID` están presentes en los tres sistemas.

Para el análisis posterior se prioriza la consistencia multifuente, conservando únicamente aquellas unidades-periodo para las que existe información financiera, de ratios y operativa.


In [86]:
#Claves para el merge, Unidad_ID y Periodo_ID, para que cada fila represente
# una unidad de negocio y un periodo de tiempo concreto

df = fin.merge(rat, on=["Unidad_ID", "Periodo_ID"], how="left")
df = df.merge(ops, on=["Unidad_ID", "Periodo_ID"], how="left")

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041 entries, 0 to 1040
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unidad_ID             1041 non-null   object 
 1   Departamento          1041 non-null   object 
 2   Periodo_ID            1041 non-null   int64  
 3   Ingresos              1041 non-null   float64
 4   Gastos                1041 non-null   float64
 5   Activos               1041 non-null   float64
 6   Pasivos               1041 non-null   float64
 7   EBIT                  1041 non-null   float64
 8   ROA                   920 non-null    float64
 9   ROE                   920 non-null    float64
 10  Margen_Explotacion    920 non-null    float64
 11  Endeudamiento         920 non-null    float64
 12  Indice_Rentabilidad   920 non-null    float64
 13  Plantilla             921 non-null    float64
 14  Costes_Fijos          921 non-null    float64
 15  CostesVariables      

In [88]:
df.head(5)

,Unidad_ID,Departamento,Periodo_ID,Ingresos,Gastos,Activos,Pasivos,EBIT,ROA,ROE,Margen_Explotacion,Endeudamiento,Indice_Rentabilidad,Plantilla,Costes_Fijos,CostesVariables,Crecimiento_Ingresos
0,UO_086,Logística,1,123456.780000,24987.379399,10860.717125,9234.423602,3811.366786,0.140907,-0.003602,0.156345,0.871875,-0.108494,270.0,5538.499759,5589.317523,0.134289
1,UO_015,Tecnología,8,30450.091226,19446.985846,17670.376360,12420.632727,12315.649015,NaN,NaN,NaN,NaN,NaN,295.0,1099.788083,2547.473093,0.101430
2,UO_077,Finanzas,1,47170.073204,33076.391477,6926.752898,2096.144509,13716.391078,0.048257,0.136341,0.315895,0.499244,0.079839,353.0,6912.371977,13080.839384,0.233546
3,UO_001,Tecnología,9,19592.915242,13880.256093,23211.886570,19841.303916,6666.729731,NaN,NaN,NaN,NaN,NaN,138.0,4808.517120,4559.273614,-0.073837
4,UO_118,Logística,2,19593.076079,13469.944561,12749.502547,4685.473804,6000.973263,0.109413,-0.016162,0.275845,0.607457,0.002040,NaN,NaN,NaN,NaN


In [89]:
df.shape

(1041, 17)

El dataset financiero se considera la fuente principal del sistema de gestión, ya que representa la existencia económica de cada unidad de negocio en un periodo dado.
Por este motivo, el proceso de unificación se realiza mediante un left join sobre los datos financieros, incorporando información de ratios y operativa cuando está disponible.
La ausencia de información en estas fuentes se interpreta como una limitación del reporting interno y se aborda posteriormente en el EDA mediante estrategias de imputación y análisis de patrones de datos faltantes.

## 7 Validación del dataset final

In [90]:

print("Shape final del dataset:", df.shape)

# Nulos por columna
print("\nNulos por columna:\n")
print(df.isnull().sum())

# Tipos de datos
print("\nTipos de datos:\n")
print(df.dtypes)

# Estadísticos básicos
print("\nEstadísticos descriptivos:\n")
df.describe()

Shape final del dataset: (1041, 17)

Nulos por columna:

Unidad_ID                 0
Departamento              0
Periodo_ID                0
Ingresos                  0
Gastos                    0
Activos                   0
Pasivos                   0
EBIT                      0
ROA                     121
ROE                     121
Margen_Explotacion      121
Endeudamiento           121
Indice_Rentabilidad     121
Plantilla               120
Costes_Fijos            120
CostesVariables         120
Crecimiento_Ingresos    120
dtype: int64

Tipos de datos:

Unidad_ID                object
Departamento             object
Periodo_ID                int64
Ingresos                float64
Gastos                  float64
Activos                 float64
Pasivos                 float64
EBIT                    float64
ROA                     float64
ROE                     float64
Margen_Explotacion      float64
Endeudamiento           float64
Indice_Rentabilidad     float64
Plantilla           

,Periodo_ID,Ingresos,Gastos,Activos,Pasivos,EBIT,ROA,ROE,Margen_Explotacion,Endeudamiento,Indice_Rentabilidad,Plantilla,Costes_Fijos,CostesVariables,Crecimiento_Ingresos
count,1041.000000,1041.000000,1041.000000,1041.000000,1041.000000,1041.000000,920.000000,920.000000,920.000000,920.000000,920.000000,921.000000,921.000000,921.000000,921.000000
mean,5.446686,25689.137997,19939.746254,21384.535097,12759.460055,5691.679567,0.064096,0.103369,0.225246,0.590972,0.006085,259.574376,6895.407707,9880.041511,0.095714
std,2.882522,14135.018629,11234.666308,14815.359452,10306.287087,4176.837617,0.050328,0.099999,0.099270,0.174328,0.058683,134.877332,4475.425909,6088.387360,0.117271
min,1.000000,5719.042519,3849.892318,2691.205621,999.075844,359.729515,-0.019836,-0.049940,0.050155,0.300032,-0.166345,20.000000,748.742557,1107.540168,-0.099318
25%,3.000000,16012.581050,12294.849808,11941.697430,6576.234146,2748.310144,0.019206,0.029859,0.139519,0.439025,-0.033187,140.000000,3900.300236,5805.449918,-0.006166
50%,5.000000,22298.062876,17058.539505,17854.909557,10168.811330,4581.079902,0.062841,0.103642,0.225317,0.585958,0.004935,264.000000,5605.281103,8314.816534,0.091201
75%,8.000000,31532.408207,24853.806017,26882.246358,15808.956791,7456.210193,0.108978,0.179988,0.309411,0.741515,0.046404,375.000000,8621.937299,12276.372000,0.202118
max,10.000000,123456.780000,89113.808154,149856.449652,113412.828749,32683.534149,0.149820,1.600000,0.399989,0.898749,0.169001,499.000000,42990.312538,48459.537917,0.299956


## 8 Guardar dataset creado 

In [92]:
# Guardar dataset limpio para el EDA
df.to_csv(r"C:\Users\balle\Desktop\DataScience\ML_project\data\Processed\dataset_clean.csv", index=False)